# Using /tags
In this example we are going to explore deeply the options available when collecting and uploading tags to EOSC Performance.

### Create the environment
To do so, we select an API endpoint and collect a token from our configuration. <br>
We also need an access token, in this example we use [oidc-agent](https://github.com/indigo-dc/oidc-agent#:~:text=oidc%2Dagent%20is%20a%20set,session%20or%20a%20login%20session.) to get one. <br>

In [1]:
eosc_perf_api="https://performance.services.fedcloud.eu/api/v1/"
access_token=$(oidc-token egi-prod)

### (Conditional) Register, if not done already
To use our service as user, first we need to accept the terms of usage and register. <br>
Make sure to read the [terms and conditions](https://performance.services.fedcloud.eu/privacy_policy).

In [ ]:
curl -X 'POST' \
  "$eosc_perf_api/users:register" \
  -H "Authorization: Bearer $access_token"

### Upload your tag
To upload the tag, you only need to use an authenticated POST request to ``/tags`` and attach the following content to the body:

- **name**: Name of the tag to display to the community.
- **description(Optional)**: Short description about the tag for the community.

In [3]:
name="my_tag"
description="A free description for the community"
curl -X 'POST' "$eosc_perf_api/tags" \
  -H 'accept: application/json' \
  -H "Authorization: Bearer $access_token" \
  -H 'Content-Type: application/json' \
  -d "{\"name\": \"$name\", \"description\": \"$description\"}" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   188  100   115  100    73    148     94 --:--:-- --:--:-- --:--:--   243
{
  "description": "A free description for the community",
  "id": "6186498a-929b-4745-9ac7-fc1d8b75b398",
  "name": "my_tag"
}


### Download your tag
You can check your tag is available by downloading the tag.

<div class="alert alert-info">
Note the upload of tags needs to be validated by administrators. Therefore it will only be available to the community after the review is completed.
</div>

In [4]:
tags=$(curl -X 'GET' "$eosc_perf_api/tags?name=$name")
echo $tags | jq '.items[0].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   205  100   205    0     0  14642      0 --:--:-- --:--:-- --:--:-- 14642
"6186498a-929b-4745-9ac7-fc1d8b75b398"


### Update results with your tags
You can use a PUT request at ``/results/{id}/tags`` to update your result tags. For example when a user has created an interesting tag that you think it might relate some of your results.<br>
You can use a GET request at ``/users/self/results`` to collect the list of results uploaded by your user. Note that you need to include your access token to use this request.

In [5]:
results=$(curl -X 'GET' "$eosc_perf_api/users/self/results" \
  -H 'accept: application/json' \
  -H "Authorization: Bearer $access_token" \
  -H 'Content-Type: application/json')
echo $results | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2081  100  2081    0     0   2627      0 --:--:-- --:--:-- --:--:--  2624
"76c5773a-52d7-408b-961b-e6d3ad06c640"


In [6]:
result_id=$(echo $results | jq -r '.items[0].id')
tag_id=$(echo $tags | jq -r '.items[0].id')
curl -X 'PUT' "$eosc_perf_api/results/$result_id/tags" \
  -H 'accept: application/json' \
  -H "Authorization: Bearer $access_token" \
  -H 'Content-Type: application/json' \
  -d "{\"tags_ids\": [\"$tag_id\"]}" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    54    0     0  100    54      0     32  0:00:01  0:00:01 --:--:--    32


<div class="alert alert-info">
You can only update tags on your uploaded results. However you can use all the community.
</div>

### Download results with your tags
With this functionality, users can easily find relevant results by just attaching the tags they are interested in into the ``/results`` query.

In [7]:
curl -X 'GET' "$eosc_perf_api/results?tags_ids=$tag_id" \
  -H 'accept: application/json' | jq '.items[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2195  100  2195    0     0   178k      0 --:--:-- --:--:-- --:--:--  178k
"76c5773a-52d7-408b-961b-e6d3ad06c640"
